# GMNS to AequilibraE example

## Inputs
Nodes as a .csv flat file in GMNS format
Links as a .csv flat file in GMNS format
Trips as a .csv flat file:  orig_node, dest_node, trips
Sqlite database used by AequilibraE

## Steps
1.  Read the GMNS nodes
-  Place in SQLite database, then translate to AequilibraE nodes
-  Generate the dictionary of zones for the omx trip table (uses node_type = centroid)

2. Read the GMNS links
-  Place in SQLite database, then translate to AequilibraE links

3. Read the trips
-  Translate into .omx file

A separate Jupyter notebook, Route, performs the following steps

4. Run AequilibraE shortest path and routing

5. Generate detail and summary outputs


In [1]:
#!/usr/bin/env python
# coding: utf-8
print('Scott Smith: Python code to extract selected zone pairs from a person trip table \n')
import os
import numpy as np
import pandas as pd
import sqlite3
#import shutil  # needed?
import openmatrix as omx
import math
run_folder = 'C:/Users/Scott.Smith/GMNS/Lima'
highest_centroid_node_number = 500

Scott Smith: Python code to extract selected zone pairs from a person trip table 



## Read the nodes, and set up the dictionary of centroids
The dictionary of centroids is used later in setting up the omx trip table

In [2]:
#Read the nodes
node_csvfile = os.path.join(run_folder, 'GMNS_node.csv')
df_node = pd.read_csv(node_csvfile)   #data already has headers
print(df_node.head())   #debugging
df_size = df_node.shape[0]
print(df_size)
#print(df_node.loc[700,"node_type"])

   node_id  name    x_coord    y_coord  z_coord node_type  ctrl_type  zone_id  \
0        1   NaN  1523373.0  1003235.0      NaN  centroid        NaN        1   
1        2   NaN  1523873.0  1003225.0      NaN  centroid        NaN        2   
2        3   NaN  1524263.0  1003205.0      NaN  centroid        NaN        3   
3        4   NaN  1524224.0  1002265.0      NaN  centroid        NaN        4   
4        5   NaN  1523854.0  1002765.0      NaN  centroid        NaN        5   

   parent_node_id  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
2232


In [17]:
# Set up the dictionary of centroids
# Assumption: the node_type = 'centroid' for centroid nodes
# The centroid nodes are the lowest numbered nodes, at the beginning of the list of nodes,
# but node numbers need not be consecutive
tazdictrow = {}
for index in df_node.index:
    if df_node['node_type'][index]=='centroid':
        #DEBUG print(index, df_node['node_id'][index], df_node['node_type'][index])
        tazdictrow[df_node['node_id'][index]]=index
#tazdictrow = {1:0,2:1,3:2,4:3,5:4,6:5,7:6,8:7,9:8,10:9,11:10,12:11,13:12,14:13,15:14,16:15,17:16,18:17,19:18,20:19,21:20,22:21,23:22,24:23,25:24,26:25,27:26,28:27,29:28,30:29,31:30,32:31,33:32,34:33,35:34,36:35,37:36,38:37,39:38,40:39,41:40,42:41,43:42,44:43,45:44,46:45,47:46,48:47,49:48,50:49,51:50,52:51,53:52,54:53,55:54,56:55,57:56,58:57,59:58,60:59,61:60,62:61,63:62,64:63,65:64,66:65,67:66,68:67,69:68,70:69,71:70,72:71,73:72,74:73,75:74,76:75,77:76,78:77,79:78,80:79,81:80,82:81,83:82,84:83,85:84,86:85,87:86,88:87,89:88,90:89,91:90,92:91,93:92,94:93,95:94,96:95,97:96,98:97,99:98,100:99,101:100,102:101,103:102,104:103,105:104,106:105,107:106,108:107,109:108,110:109,111:110,112:111,113:112,114:113,115:114,116:115,117:116,118:117,119:118,120:119,121:120,122:121,123:122,124:123,125:124,126:125,127:126,128:127,129:128,130:129,131:130,132:131,133:132,134:133,135:134,136:135,137:136,138:137,139:138,140:139,141:140,142:141,143:142,144:143,145:144,146:145,147:146,148:147,149:148,150:149,151:150,152:151,153:152,154:153,155:154,156:155,157:156,158:157,159:158,160:159,161:160,162:161,163:162,164:163,165:164,166:165,167:166,168:167,169:168,170:169,171:170,172:171,173:172,174:173,175:174,176:175,177:176,178:177,179:178,180:179,181:180,182:181,183:182,184:183,185:184,186:185,187:186,188:187,189:188,190:189,191:190,192:191,193:192,194:193,195:194,196:195,197:196,198:197,199:198,200:199,201:200,202:201,203:202,204:203,205:204,206:205,207:206,208:207,209:208,210:209,211:210,212:211,213:212,214:213,215:214,216:215,217:216,218:217,219:218,220:219,221:220,222:221,223:222,224:223,225:224,226:225,227:226,228:227,229:228,230:229,231:230,232:231,233:232,234:233,235:234,236:235,237:236,238:237,239:238,240:239,241:240,242:241,243:242,244:243,245:244,246:245,247:246,248:247,249:248,250:249,251:250,252:251,253:252,254:253,255:254,256:255,257:256,258:257,259:258,260:259,261:260,262:261,263:262,264:263,265:264,266:265,267:266,268:267,269:268,270:269,271:270,272:271,273:272,274:273,275:274,276:275,277:276,278:277,279:278,280:279,281:280,282:281,283:282,284:283,285:284,286:285,287:286,288:287,289:288,290:289,291:290,292:291,293:292,294:293,295:294,296:295,297:296,298:297,299:298,300:299,301:300,302:301,303:302,304:303,305:304,306:305,307:306,308:307,309:308,310:309,311:310,312:311,313:312,314:313,315:314,316:315,317:316,318:317,319:318,320:319,321:320,322:321,323:322,324:323,325:324,326:325,327:326,328:327,329:328,330:329,331:330,332:331,333:332,334:333,335:334,336:335,337:336,338:337,339:338,340:339,341:340,342:341,343:342,344:343,345:344,346:345,347:346,348:347,349:348,350:349,351:350,352:351,353:352,354:353,355:354,356:355,357:356,358:357,359:358,360:359,361:360,362:361,363:362,364:363,365:364,366:365,367:366,368:367,369:368,370:369,371:370,372:371,373:372,374:373,375:374,376:375,377:376,378:377,379:378,380:379,381:380,382:381,383:382,384:383,385:384,386:385,387:386,388:387,389:388,390:389,391:390,392:391,393:392,394:393,395:394,440:395,441:396,442:397,443:398,444:399,445:400,446:401,447:402,448:403,449:404,450:405,451:406,452:407,453:408,454:409,455:410,456:411,457:412,458:413,459:414,460:415,461:416,462:417,463:418,464:419,465:420,466:421,467:422,468:423,469:424,470:425,471:426,472:427,473:428,474:429,475:430,476:431,477:432,478:433,479:434,480:435,481:436,482:437,483:438,484:439,485:440,486:441,487:442,488:443,489:444,490:445,491:446,492:447,493:448}
taz_list = list(tazdictrow.keys())
matrix_size = len(tazdictrow)   #Matches the number of nodes flagged as centroids
print(matrix_size)   #DEBUG
highest_centroid_node_number = max(tazdictrow, key=tazdictrow.get)  #for future use
print(highest_centroid_node_number)  #DEBUG

449
493


## Read the links

In [4]:
# Read the links
link_csvfile = os.path.join(run_folder, 'GMNS_link.csv')
df_link = pd.read_csv(link_csvfile)   #data already has headers
#print(df_node.head())   #debugging
#df_size = df_link.shape[0]
print(df_link.shape[0])  #debug

6095


## Put nodes and links into SQLite.  Then translate to AequilibraE 0.6.5 format
1. Nodes are pushed into a table named GMNS_node
2. node table used by AequilibraE is truncated, then filled with values from GMNS_node
3. Centroid nodes are assumed to be the lowest numbered nodes, limited by the highest_centroid_node_number
    -  Number of centroid nodes must equal matrix_size, the size of the trip OMX Matrix
3. Links are pushed into a table named GMNS_link
4. link table used by AequilibraE is truncated, then filled with values from GMNS_link

In [5]:
#Open the Sqlite database, and insert the nodes and links
network_db = os.path.join(run_folder,'1_project','Lima.sqlite')
with sqlite3.connect(network_db) as db_con:
    #nodes
    df_node.to_sql('GMNS_node',db_con, if_exists='replace',index=False)
    db_cur = db_con.cursor()
    sql0 = "delete from nodes;"
    db_cur.execute(sql0)
    sql1 = ("insert into nodes(ogc_fid, node_id, x, y, is_centroid)" +
            " SELECT node_id, node_id, x_coord,y_coord,0 from " +
            " GMNS_node")
    db_cur.execute(sql1)
    sql2 = ("update nodes set is_centroid = 1 where ogc_fid <= " + str(highest_centroid_node_number))
    db_cur.execute(sql2)
    
with sqlite3.connect(network_db) as db_con:
    df_link.to_sql('GMNS_link',db_con, if_exists='replace',index=False)
    db_cur = db_con.cursor()
    sql0 = "delete from links;"
    db_cur.execute(sql0)
    sql1 = ("insert into links(ogc_fid, link_id, a_node, b_node, direction, distance, modes," +
            " link_type, capacity_ab, speed_ab, free_flow_time) " +
            " SELECT link_id, link_id, from_node_id, to_node_id, directed, length, allowed_uses," +
            " facility_type, capacity, free_speed, 60*length / free_speed" +
            " FROM GMNS_link where GMNS_link.capacity > 0")
    db_cur.execute(sql1)
    sql2 = ("update links set capacity_ba = 0, speed_ba = 0, b=0.15, power=4")
    db_cur.execute(sql2)

## Read the trips, and translate to omx file

In [6]:
#Read a flat file trip table into pandas dataframe
trip_csvfile = os.path.join(run_folder, 'demand.csv')
df_trip = pd.read_csv(trip_csvfile)   #data already has headers
print(df_trip.head())   #debugging
df_size = df_trip.shape[0]
print(df_size)
#print(df.iloc[50]['o_zone_id'])
#stuff for debugging
for k in range(df_size):   #at most matrix_size*matrix_size
    i = tazdictrow[df_trip.iloc[k]['orig_taz']]
    j = tazdictrow[df_trip.iloc[k]['dest_taz']]
    if k == 4:  print(k,"  i=",i,"  j=",j)

   orig_taz  dest_taz  total
0         1        57      1
1         1       138      1
2         2         2      1
3         2       287      1
4         3        67      1
13000
4   i= 2   j= 66


In [7]:
#Write the dataframe to an omx file
# This makes use of tazdictrow, and matrix_size, that was established earlier. 
# The rows are written to a file that is used only for debugging
outfile = os.path.join(run_folder, '0_tntp_data' ,'demand.omx') 
outdebugfile = open(os.path.join(run_folder,'debug_demand.txt'),"w")
output_demand = np.zeros((matrix_size,matrix_size))
f_output = omx.open_file(outfile,'w')

f_output.create_mapping('taz',taz_list)
#write the data
for k in range(df_size):   #at most matrix_size*matrix_size
    i = tazdictrow[df_trip.iloc[k]['orig_taz']]
    j = tazdictrow[df_trip.iloc[k]['dest_taz']]
  
    output_demand[i][j] = df_trip.iloc[k]['total']
    print('Row: ',df_trip.iloc[k]['orig_taz'],i,"  Col: ",df_trip.iloc[k]['dest_taz'],j," Output",output_demand[i][j],file=outdebugfile)
   
f_output['matrix'] = output_demand
f_output.close()
outdebugfile.close()
#Stop here

In [8]:
# Not needed except for debugging
#Read the input omx trip table
infile = os.path.join(run_folder, '0_tntp_data' ,'demand.omx') 
f_input = omx.open_file(infile)
m1 = f_input['matrix']
input_demand = np.array(m1)

print('Shape:',f_input.shape())
print('Number of tables',len(f_input))
print('Table names:',f_input.list_matrices())
print('attributes:',f_input.list_all_attributes())
print('sum of trips',np.sum(m1))

Shape: (449, 449)
Number of tables 1
Table names: ['matrix']
attributes: []
sum of trips 32041.0


In [9]:
#input_demand.sum(axis=0)

In [10]:
#input_demand.sum(axis=1)

In [11]:
#for i in range(matrix_size):
#    for j in range(matrix_size):
#        print(i+1,j+1,input_demand[i][j])

In [12]:
f_output.close()